In [160]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.checkpoint.sqlite import SqliteSaver

import dotenv
dotenv.load_dotenv()

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,
)

config = {
    "configurable" : {
        "thread_id": "0_x"
    }
}

In [161]:
class State(MessagesState):
    pass

graph_builder = StateGraph(State)

In [162]:

def chatbot(state: State):
    response = llm.invoke(
        state["messages"]
    )
    return {"messages":[response]}


In [163]:

graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn)
)


In [164]:
result = graph.invoke(
    {
        "messages" : [
            {
                "role":"user",
                "content":"I live in Korea now. And the city I live in is Seoul.",
            },
        ]
    },
    config = config,
)

In [165]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I live in Korea now. And the city I live in is Seoul.
================================== Ai Message ==================================

That's great! Seoul is a vibrant city with a rich history and a dynamic mix of modern and traditional culture. Whether you're interested in exploring historical sites like Gyeongbokgung Palace, enjoying the bustling atmosphere of Myeongdong, or experiencing the nightlife in Hongdae, there's always something to see and do. How are you finding life in Seoul so far?


In [166]:
state_history = graph.get_state_history(config)

In [167]:
for state_snapshot in list(state_history):
    print(state_snapshot.next)
    print(state_snapshot.values["messages"])
    print("=========\n")

()
[HumanMessage(content='I live in Korea now. And the city I live in is Seoul.', additional_kwargs={}, response_metadata={}, id='49e28cc3-8b41-460c-afdb-b3f548459a29'), AIMessage(content="That's great! Seoul is a vibrant city with a rich history and a dynamic mix of modern and traditional culture. Whether you're interested in exploring historical sites like Gyeongbokgung Palace, enjoying the bustling atmosphere of Myeongdong, or experiencing the nightlife in Hongdae, there's always something to see and do. How are you finding life in Seoul so far?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 22, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1590f93f9d', 'id': 'chatcmpl-D3Ft

In [169]:
state_history = graph.get_state_history(config)

to_fork = list(state_history)[-2]

to_fork.config

{'configurable': {'thread_id': '0_x',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0fcdc8-3e9b-68f4-8000-4d2aa47fbdd7'}}

In [170]:
from langchain_core.messages import HumanMessage

graph.update_state(
    to_fork.config,
    {
        "messages" : [
            HumanMessage(
                content="I live in Korea now. And the city I live in is Busan.",
                id="1f0fcdc8-3e9b-68f4-8000-4d2aa47fbdd7",
            )
        ]
    }
)

ImportError: cannot import name 'MumanMessage' from 'langchain_core.messages' (/Users/askstoryteam/Documents/Obsidian Vault/Project/AI Agent/course/hello-langgraph/.venv/lib/python3.13/site-packages/langchain_core/messages/__init__.py)